Credits:

- https://www.kaggle.com/code/itsuki9180/detect-sleep-states-dataprepare  
- https://www.kaggle.com/code/werus23/sleep-critical-point-prepare-data

GO TO:
Data Preparation - this
TRAIN - https://www.kaggle.com/code/jhjh97/transformer-train
INFERENCE - Not Yet

This does:
- Save preprocessed gaussian outputs according to following parameters `N_DAYS_PER_SAMPLE`, `SEPERATE_FEATURES`.
- `N_DAYS_PER_SAMPLE`(int) is how many days you like to windowing for model.
- `SEPERATE_FEATURES`(bool) whether you split your data for seperated model.
- `SIGMA` for gaussian sigma value(std). more smaller SIGMA is, more sharper distribution gets.

## Ver1.1

- Split hard sample and easy samples

In [ ]:
N_DAYS_PER_SAMPLE = 3
FIT_INTO_1DAY = True
SEPERATE_FEATURES = True  # Not Used
# GENERATE_FEATURES = True # generate in Train torch.data.DataSet for efficiency

SIGMA = 720 # target gaussian parameter, 12 * 60

In [ ]:
import pandas as pd
import numpy as np
import gc
import time
import json
from datetime import datetime
import matplotlib.pyplot as plt
import os
from os.path import join, exists
import joblib
import random
import math
from tqdm.auto import tqdm 
import shutil

from scipy.interpolate import interp1d

from math import pi, sqrt, exp
import sklearn,sklearn.model_selection
import torch
from torch import nn,Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from sklearn.metrics import average_precision_score
from timm.scheduler import CosineLRScheduler
plt.style.use("ggplot")

from pyarrow.parquet import ParquetFile
import pyarrow as pa 
import ctypes

In [ ]:
out_dir = "train_data"
os.makedirs(out_dir, exist_ok = True)

In [ ]:
class PATHS:
    MAIN_DIR = "/kaggle/input/child-mind-institute-detect-sleep-states/"
    # CSV FILES : 
    TRAIN_EVENTS = MAIN_DIR + "train_events.csv"
    # PARQUET FILES:
    TRAIN_SERIES = MAIN_DIR + "train_series.parquet"
    SAVE_DIR = out_dir

In [ ]:
def preprocess_events(data_path, save_dir):
    """ events.csv """
    events = pd.read_csv(data_path)
    if not exists(join(save_dir, 'id_map.parquet')):
        id_map = pd.DataFrame({"series_id": events.series_id.unique(),
                                "id_index": [i for i in range(len(events.series_id.unique()))]})
        id_map.id_index = id_map.id_index.astype(np.uint16)
        id_map.to_parquet(os.path.join(save_dir,"id_map.parquet"), index=False)
    id_map = pd.read_parquet(os.path.join(save_dir,"id_map.parquet"))
    events.dropna(subset=['step'], inplace=True)
    # reduce memory size + event re-label
    events = events.merge(right=id_map, on="series_id").drop(columns="series_id")
    events.step = events.step.astype(np.uint32)
    events.night = events.night.astype(np.uint16)
#     events.to_csv(os.path.join(save_dir,'preprocessed_events.csv'), index=False)
    
    return events

In [ ]:
def preprocess_series(data_path, save_dir):
    """ train_series.parquet """
    # reduce memory
    series = pd.read_parquet(data_path)
    if not exists(join(save_dir, 'id_map.parquet')):
        id_map = pd.DataFrame({"series_id": series.series_id.unique(),
                                "id_index": [i for i in range(len(series.series_id.unique()))]})
        id_map.id_index = id_map.id_index.astype(np.uint16)
        id_map.to_parquet(os.path.join(save_dir,"id_map.parquet"), index=False)
    
    id_map = pd.read_parquet(os.path.join(save_dir,"id_map.parquet"))
    series = series.merge(right=id_map, on='series_id').drop(columns='series_id').reset_index()
    series.step = series.step.astype(np.uint32)

    # normalize enmo and anglez
    mean_enmo = series['enmo'].mean()
    std_enmo = series['enmo'].std()
    series.enmo = (series['enmo'] - mean_enmo)/std_enmo
    mean_anglez = series['anglez'].mean()
    std_anglez = series['anglez'].std()
    series.anglez = (series['anglez'] - mean_anglez)/std_anglez
    
    series = series.drop(columns='index')
#     series.to_parquet(os.path.join(save_dir,'preprocessed_series.parquet'))
    
    return series

In [ ]:
# must process events first.
events = preprocess_events(PATHS.TRAIN_EVENTS, PATHS.SAVE_DIR)
series = preprocess_series(PATHS.TRAIN_SERIES, PATHS.SAVE_DIR)

converting evetns into gaussian target numpy array.

In [ ]:
def gauss(n=SIGMA,sigma=SIGMA*0.15):
    # guassian distribution function
    r = range(-int(n/2),int(n/2)+1)  # int는 나머지 버림계산
    return [1 / (sigma * sqrt(2*pi)) * exp(-float(x)**2/(2*sigma**2)) for x in r]

In [ ]:
def get_pad_amount(series):
    # For fitting data into 24 hours cycle, get how much series need to be zero_padded for front and end.
    # 0.4145 secs
    f_h=series.timestamp.dt.hour[0]
    f_m=series.timestamp.dt.minute[0]
    f_s=series.timestamp.dt.second[0]
    r_h=list(series.timestamp.dt.hour)[-1]
    r_m=list(series.timestamp.dt.minute)[-1]
    r_s=list(series.timestamp.dt.second)[-1]
    
    # calc how many steps first data is apart from 0:00:00.
    front = f_s//5 + f_m*12 + f_h*(12*60)
    if front>0:
        front -= 1
    
    # calc how many steps rear data is apart from 23:59:55
    max_step = 24*60*12
    end = max_step - (r_s//5 + r_m*12 + r_h*(12*60))
    
    return front, end

# New 2

### can i reduce data size? -> Y

- 어차피 conv 계열만 쓰기에, X 임베딩을 0-24로 맞출 필요가 없음.
- 하지만, 잠들기 전 후로 데이터에 증거가 많이 남아있을 것 같음. 그래서 (전날 오후 6시 ~ 당일 ~ 다음날 오전 6시)로 시간을 맞춰줄 예정. 
- 24+6+6=36시간 = 25920, /60=432  

#### easy, hard(unlabelded)로 나눠주기

- 일단 easy로 학습시키고 모델이 얼마나 잘 성능 나오는지 보고싶음
- easy(train,val), mixed(train,val), hard(tarin,val)
- 기준은 하루동안 label이 없으면 hard example

In [ ]:
data_counter=0
enmo_x=[]
enmo_y=[]
attach_hours=6
ids = pd.read_parquet(os.path.join(PATHS.SAVE_DIR,"id_map.parquet"))

if SEPERATE_FEATURES:
    os.makedirs(PATHS.SAVE_DIR+'/enmo/hard', exist_ok = True)
    os.makedirs(PATHS.SAVE_DIR+'/enmo/fine', exist_ok = True)
    os.makedirs(PATHS.SAVE_DIR+'/anglez/hard', exist_ok = True)
    os.makedirs(PATHS.SAVE_DIR+'/anglez/fine', exist_ok = True)

# for ids, make "series + target_gaussian" vector and store them ids by ids.
for cur_id in tqdm(ids.id_index, total=len(ids)):
    cur_targets = []
    cur_events = events[events.id_index == cur_id].copy()
    cur_series = series.loc[(series.id_index == cur_id)].copy().reset_index(drop=True)
    cur_series['timestamp'] = pd.to_datetime(cur_series.timestamp,format = '%Y-%m-%dT%H:%M:%S%z').astype("datetime64[ns, UTC-04:00]")
    
    # calculate how many steps needed to pad for each series
    front, end = get_pad_amount(cur_series)
    cur_events.step += front
    total_len = len(cur_series)+front+end
    target_gaussian = np.zeros((total_len,2))  # onset:0, wakeup:1
    for i in range(len(cur_events)):
        s=cur_events.iloc[i].step
        gau_st = max(SIGMA//2-s, 0)
        gau_ed = SIGMA+(SIGMA+1)%2+1 - max(s+SIGMA//2-(len(target_gaussian)-1), 0)
        tar_st = max(0, s-SIGMA//2)
        tar_ed = min(len(target_gaussian), s+SIGMA//2+1)
        if cur_events.iloc[i].event=='onset':
            target_gaussian[tar_st:tar_ed,0] = gauss()[gau_st:gau_ed]
        if cur_events.iloc[i].event=='wakeup':
            target_gaussian[tar_st:tar_ed,1] = gauss()[gau_st:gau_ed]

    # zero padding for series
    cur_series = cur_series.drop(columns=['id_index','step','timestamp'])
    zero_pad_front = pd.DataFrame({'anglez':[0 for _ in range(front)], 'enmo':[0 for _ in range(front)]}, dtype=np.float32)
    zero_pad_end = pd.DataFrame({'anglez':[0 for _ in range(end)], 'enmo':[0 for _ in range(end)]}, dtype=np.float32)
    cur_series = pd.concat([zero_pad_front, cur_series, zero_pad_end], ignore_index=True)
    del zero_pad_front, zero_pad_end
    gc.collect()
    
    # normalize target into max value.
    target_gaussian /= np.max(target_gaussian + 1e-12)
    
    # append target onto series,
    cur_series['onset'] = target_gaussian[:,0]
    cur_series['wakeup'] = target_gaussian[:,1]
    
    # save it into croped size
    one_day_steps = (24*60*12)
    cur_series_days = len(cur_series)//one_day_steps
    for i in range(cur_series_days - N_DAYS_PER_SAMPLE + 1):
        st = i*one_day_steps
        ed = (i+N_DAYS_PER_SAMPLE)*one_day_steps
        st2=(24-attach_hours)*60*12
        ed2=-(24-attach_hours)*60*12
        enmo_sample = cur_series[st:ed].drop(columns='anglez')[st2:ed2]
        
        if sum(enmo_sample['onset'])==0.0 and sum(enmo_sample['wakeup'])==0.0:
            # label없는 seq는 hard에 저장
            # plotting(enmo_sample)
            enmo_sample.to_csv(f'{PATHS.SAVE_DIR}/enmo/hard/enmo_id{cur_id}_{data_counter}.csv', index=False)
        else:
            # onset, wakeup 하나라도 있으면 fine에 저장
            enmo_sample.to_csv(f'{PATHS.SAVE_DIR}/enmo/fine/enmo_id{cur_id}_{data_counter}.csv', index=False)
        anglez_sample = cur_series[st:ed].drop(columns='enmo')
        if sum(anglez_sample['onset'])==0.0 and sum(anglez_sample['wakeup']==0.0):
            anglez_sample.to_csv(f'{PATHS.SAVE_DIR}/anglez/hard/anglez_id{cur_id}_{data_counter}.csv', index=False)
        else:
            anglez_sample.to_csv(f'{PATHS.SAVE_DIR}/anglez/fine/anglez_id{cur_id}_{data_counter}.csv', index=False)
        data_counter+=1
    del cur_series, cur_events, target_gaussian, enmo_sample, anglez_sample
    gc.collect()

# NEW

- including 8 hours only front, back
- feature engineering included
- gaussian label
- directly store 28GB data


#### ISSUE

- "failed" because of memory shortage

In [ ]:
# def generate_features_enmo(xy):
#     for r in [17, 33, 65]:
#         tmp_feat = xy['enmo'].rolling(r, center=True)
#         xy[f'enmo_mean_{r}'] = tmp_feat.mean()
#         xy[f'enmo_std_{r}'] = tmp_feat.std()
#     return xy.drop(columns=['onset','wakeup']).fillna(0)

In [ ]:
# data_counter=0
# enmo_x=[]
# enmo_y=[]
# ids = pd.read_parquet(os.path.join(PATHS.SAVE_DIR,"id_map.parquet"))

# if SEPERATE_FEATURES:
#     os.makedirs(PATHS.SAVE_DIR+'/enmo', exist_ok = True)
#     os.makedirs(PATHS.SAVE_DIR+'/anglez', exist_ok = True)

    
# # for ids, make "series + target_gaussian" vector and store them ids by ids.
# for idx, cur_id in tqdm(enumerate(ids.id_index), total=len(ids)):
#     cur_targets = []
#     cur_events = events[events.id_index == cur_id].copy()
#     cur_series = series.loc[(series.id_index == cur_id)].copy().reset_index(drop=True)
#     cur_series['timestamp'] = pd.to_datetime(cur_series.timestamp,format = '%Y-%m-%dT%H:%M:%S%z').astype("datetime64[ns, UTC-04:00]")
    
#     # calculate how many steps needed to pad for each series
#     front, end = get_pad_amount(cur_series)
#     cur_events.step += front
#     total_len = len(cur_series)+front+end
#     target_gaussian = np.zeros((total_len,2))  # onset:0, wakeup:1
    
#     for i in range(len(cur_events)):
#         s=cur_events.iloc[i].step
#         gau_st = max(SIGMA//2-s, 0)
#         gau_ed = SIGMA+(SIGMA+1)%2+1 - max(s+SIGMA//2-(len(target_gaussian)-1), 0)
#         tar_st = max(0, s-SIGMA//2)
#         tar_ed = min(len(target_gaussian), s+SIGMA//2+1)
#         if cur_events.iloc[i].event=='onset':
#             target_gaussian[tar_st:tar_ed,0] = gauss()[gau_st:gau_ed]
#         if cur_events.iloc[i].event=='wakeup':
#             target_gaussian[tar_st:tar_ed,1] = gauss()[gau_st:gau_ed]
    
#     print(cur_series)
#     break
#     # zero padding for series
#     cur_series = cur_series.drop(columns=['id_index','step','timestamp'])
#     zero_pad_front = pd.DataFrame({'anglez':[0 for _ in range(front)], 'enmo':[0 for _ in range(front)]}, dtype=np.float32)
#     zero_pad_end = pd.DataFrame({'anglez':[0 for _ in range(end)], 'enmo':[0 for _ in range(end)]}, dtype=np.float32)
#     cur_series = pd.concat([zero_pad_front, cur_series, zero_pad_end], ignore_index=True)
#     del zero_pad_front, zero_pad_end
#     gc.collect()
    
#     # normalize target into max value.
#     target_gaussian /= np.max(target_gaussian + 1e-12)
    
#     # append target onto series, save it.
#     cur_series['onset'] = target_gaussian[:,0]
#     cur_series['wakeup'] = target_gaussian[:,1]
    
#     one_day_steps = (24*60*12)
#     cur_series_days = len(cur_series)//one_day_steps
#     for i in range(cur_series_days - N_DAYS_PER_SAMPLE + 1):
#         st = i*one_day_steps
#         ed = (i+N_DAYS_PER_SAMPLE)*one_day_steps
#         if SEPERATE_FEATURES:
#             st2=(24-attach_hours)*60*12
#             ed2=-(24-attach_hours)*60*12
#             anglez.append(cur_series[st:ed].drop(columns='enmo')[st2:ed2].to_numpy())
#             enmo_xy = cur_series[st:ed].drop(columns='anglez')
#             x = generate_features_enmo(enmo_xy).to_numpy()[st2:ed2]
#             y = enmo_xy[['onset','wakeup']].to_numpy()[st2:ed2]
#             enmo_x.append(x)
#             enmo_y.append(y)
#         else:
#             pass
#         data_counter+=1
        
#     del cur_series, cur_events, target_gaussian
#     gc.collect()

# PREV

In [ ]:
# targets = []
# data = []
# data_counter=0
# ids = pd.read_parquet(os.path.join(PATHS.SAVE_DIR,"id_map.parquet"))

# if SEPERATE_FEATURES:
#     os.makedirs(PATHS.SAVE_DIR+'/enmo', exist_ok = True)
#     os.makedirs(PATHS.SAVE_DIR+'/anglez', exist_ok = True)

# # for ids, make "series + target_gaussian" vector and store them ids by ids.
# for cur_id in tqdm(ids.id_index, total=len(ids)):
#     cur_targets = []
#     cur_events = events[events.id_index == cur_id].copy()
#     cur_series = series.loc[(series.id_index == cur_id)].copy().reset_index(drop=True)
#     cur_series['timestamp'] = pd.to_datetime(cur_series.timestamp,format = '%Y-%m-%dT%H:%M:%S%z').astype("datetime64[ns, UTC-04:00]")
    
#     # calculate how many steps needed to pad for each series
#     front, end = get_pad_amount(cur_series)
#     cur_events.step += front
#     total_len = len(cur_series)+front+end
#     target_gaussian = np.zeros((total_len,2))  # onset:0, wakeup:1
#     for i in range(len(cur_events)):
#         s=cur_events.iloc[i].step
#         gau_st = max(SIGMA//2-s, 0)
#         gau_ed = SIGMA+(SIGMA+1)%2+1 - max(s+SIGMA//2-(len(target_gaussian)-1), 0)
#         tar_st = max(0, s-SIGMA//2)
#         tar_ed = min(len(target_gaussian), s+SIGMA//2+1)
#         if cur_events.iloc[i].event=='onset':
#             target_gaussian[tar_st:tar_ed,0] = gauss()[gau_st:gau_ed]
#         if cur_events.iloc[i].event=='wakeup':
#             target_gaussian[tar_st:tar_ed,1] = gauss()[gau_st:gau_ed]

#     # zero padding for series
#     cur_series = cur_series.drop(columns=['id_index','step','timestamp'])
#     zero_pad_front = pd.DataFrame({'anglez':[0 for _ in range(front)], 'enmo':[0 for _ in range(front)]}, dtype=np.float32)
#     zero_pad_end = pd.DataFrame({'anglez':[0 for _ in range(end)], 'enmo':[0 for _ in range(end)]}, dtype=np.float32)
#     cur_series = pd.concat([zero_pad_front, cur_series, zero_pad_end], ignore_index=True)
#     del zero_pad_front, zero_pad_end
#     gc.collect()
    
#     # normalize target into max value.
#     target_gaussian /= np.max(target_gaussian + 1e-12)
    
#     # append target onto series, save it.
#     cur_series['onset'] = target_gaussian[:,0]
#     cur_series['wakeup'] = target_gaussian[:,1]
    
#     one_day_steps = (24*60*12)
#     cur_series_days = len(cur_series)//one_day_steps
#     for i in range(cur_series_days - N_DAYS_PER_SAMPLE + 1):
#         st = i*one_day_steps
#         ed = (i+N_DAYS_PER_SAMPLE)*one_day_steps
#         if SEPERATE_FEATURES:
#             cur_series[st:ed].drop(columns='anglez').copy().to_csv(f'{PATHS.SAVE_DIR}/enmo/enmo_id{cur_id}_{data_counter}.csv', index=False)
#             cur_series[st:ed].drop(columns='enmo').copy().to_csv(f'{PATHS.SAVE_DIR}/anglez/anglez_id{cur_id}_{data_counter}.csv', index=False)
#         else:
#             pass
#             # cur_series[st:ed].copy().to_csv(f'{PATHS.SAVE_DIR}/id{cur_id}_total{data_counter}.csv', index=False)
#         data_counter+=1
        
#     del cur_series, cur_events, target_gaussian
#     gc.collect()

## dir move

In [ ]:
# Zip file. Because amount of data is over 50.
shutil.move(f'{PATHS.SAVE_DIR}/id_map.parquet', './')
events[:3].copy().to_csv('./dummy.csv')

if SEPERATE_FEATURES:
    os.makedirs('./enmo', exist_ok = True)
    os.makedirs('./anglez', exist_ok = True)
    shutil.make_archive('./enmo', 'zip', f'{PATHS.SAVE_DIR}/enmo')
    shutil.make_archive('./anglez', 'zip', f'{PATHS.SAVE_DIR}/anglez')
    shutil.rmtree(f'{PATHS.SAVE_DIR}')
    shutil.rmtree(f'./enmo')
    shutil.rmtree(f'./anglez')
else:
    pass

# check

In [ ]:
def plotting(sample):
    plt.figure(figsize=(16,6), dpi=100)
    plt.subplot(211)
    plt.plot(sample.drop(columns=['onset','wakeup']), 'navy', linewidth=0.5)
    plt.xlabel('enmo/anglez')
    
    plt.subplot(212)
    plt.plot(sample.onset, 'r--')
    plt.plot(sample.wakeup, 'b--')
    plt.show()

In [ ]:
enmo_len_ck_path = '/kaggle/working/train_data/enmo/fine/enmo_id4_128.csv'
sample = pd.read_csv(enmo_len_ck_path)
print(len(sample))

plotting(sample)

In [ ]:
# enmo_fine1='/kaggle/working/train_data/enmo/fine/enmo_id3_99.csv'
# enmo_fine2='/kaggle/working/train_data/enmo/fine/enmo_id4_121.csv'
# enmo_fine3='/kaggle/working/train_data/enmo/fine/enmo_id6_193.csv'

# enmo_hard1='/kaggle/working/train_data/enmo/hard/enmo_id3_120.csv'
# enmo_hard2='/kaggle/working/train_data/enmo/hard/enmo_id1_45.csv'
# enmo_hard3='/kaggle/working/train_data/enmo/hard/enmo_id5_158.csv'

# af1='/kaggle/working/train_data/anglez/fine/enmo_id16_450.csv'
# af2='/kaggle/working/train_data/anglez/fine/enmo_id10_273.csv'
# af3='/kaggle/working/train_data/anglez/fine/enmo_id6_182.csv'

# ah1='/kaggle/working/train_data/anglez/hard/enmo_id6_197.csv'
# ah2='/kaggle/working/train_data/anglez/hard/enmo_id4_141.csv'
# ah3='/kaggle/working/train_data/anglez/hard/enmo_id1_45.csv'

In [ ]:
# sample = pd.read_csv(enmo_fine1)
# plotting(sample)

In [ ]:
# sample = pd.read_csv(enmo_fine2)
# plotting(sample)

In [ ]:
# sample = pd.read_csv(enmo_fine3)
# plotting(sample)

In [ ]:
# sample = pd.read_csv(af1)
# plotting(sample)

In [ ]:
# sample = pd.read_csv(af2)
# plotting(sample)

In [ ]:
# sample = pd.read_csv(af3)
# plotting(sample)

hard

In [ ]:
# sample = pd.read_csv(enmo_hard1)
# plotting(sample)

In [ ]:
# sample = pd.read_csv(enmo_hard2)
# plotting(sample)

In [ ]:
# sample = pd.read_csv(enmo_hard3)
# plotting(sample)

In [ ]:
# sample = pd.read_csv(ah1)
# plotting(sample)

In [ ]:
# sample = pd.read_csv(ah2)
# plotting(sample)

In [ ]:
# sample = pd.read_csv(ah3)
# plotting(sample)